# Pourquoi et comment

**Objectif** : recueillir les pnn (càd les identifiants IdRef) des contributeurs à la revue.

**Démarche** : 
1) Structurer la des prénoms / noms avec colonnes "NOM"et "Prenom"
2) Générer l'url idRef pour chaque nom. paramètres utilisées:
    * 
3) Extraire le nbr d'occurences du nom 

**Documentation**: Recours à l'API SolR<a href = "http://documentation.abes.fr/aideidrefdeveloppeur/index.html"> abes </a> 


In [1]:
import re
import pandas as pd
import csv
import numpy as np 
import requests
from bs4 import BeautifulSoup
import lxml

## 1. Structurer la liste des nom/prénoms

In [2]:
with open ("../BSPE/Membres_SPE/Liste-Noms-Contributeurs.csv") as contributeurs:
    df_contributeurs = pd.read_csv(contributeurs)

In [3]:
df_contributeurs

,NOM (Prenom)
0,ACHALME (P.)
1,ADDE (R.)
2,AHMED (E.)
3,ALARY (A.)
4,ALLAIN (Jacques)
...,...
658,YVERNAULT (A.)
659,YVON (P.)
660,ZABOLOTNY
661,ZAMMIT (TH.)


In [4]:
## Séparer NOM et Prénom : 

#Nouvelle colonne pour le NOM => .iloc[i,1]
df_contributeurs["NOM"] = df_contributeurs.apply(lambda _: '', axis=1)
#Nouvelle colonne pour le PRENOM => .iloc[i,2]
df_contributeurs["Prenom"] = df_contributeurs.apply(lambda _: '', axis=1)


#Regex NOM (Prénom)
reNom = r'([-\'A-zÀ-ÿ]+((-| |\’)[A-zÀ-ÿ]+)?)( \(.*)?'
rePrenom = r'\((.*)(\.)?\)'

for i in range(len(df_contributeurs)):
    
    df_contributeurs.iloc[i,1] = re.search(reNom,df_contributeurs.iloc[i,0]).group(1)
    Prenom = re.search(rePrenom,df_contributeurs.iloc[i,0])
    if Prenom:
        Prenom2=re.sub(r' {1,2}',"",Prenom.group(1))
        df_contributeurs.iloc[i,2]=Prenom2
   

In [5]:
df_contributeurs

,NOM (Prenom),NOM,Prenom
0,ACHALME (P.),ACHALME,P.
1,ADDE (R.),ADDE,R.
2,AHMED (E.),AHMED,E.
3,ALARY (A.),ALARY,A.
4,ALLAIN (Jacques),ALLAIN,Jacques
...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.
659,YVON (P.),YVON,P.
660,ZABOLOTNY,ZABOLOTNY,
661,ZAMMIT (TH.),ZAMMIT,TH.


In [6]:
df_contributeurs.iloc[158,1]

'D’HÉRELLE'

## 2. Générer les url idref et le entrer dans le df

In [7]:
#Ajout de colonnes au df pour les url. 
df_contributeurs = df_contributeurs.reindex(columns = df_contributeurs.columns.tolist() + ['Recherche_G_t','Recherche_NomPrenom_t'])
RechercheG = []
RechercheNOMPrenom =[]

In [8]:
for i in range(len(df_contributeurs)):
#     print(df_contributeurs.iloc[i,0])
    NOM=df_contributeurs.iloc[i,1]
    Prenom=df_contributeurs.iloc[i,2]
    
    #Paramétrage : 
    ## => recordtype_z :h (càd Auteur Titre)
    
    
    if re.search(r'(.*)' '(.*)',NOM):#Si nom composé. Ex : "DE MAURICOURT"
        NOM2=NOM.replace(' ','%20AND%20')
        
        if Prenom:

            #url générale:
            df_contributeurs.iloc[i,3]="https://www.idref.fr/Sru/Solr?q=persname_t:"+NOM2+"%20"+Prenom+"&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
            RechercheG.append(df_contributeurs.iloc[i,3])

            #url NOM AND Prenom:
            df_contributeurs.iloc[i,4] ="https://www.idref.fr/Sru/Solr?q=persname_t:("+NOM2+"%20AND%20"+Prenom+")&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
            RechercheNOMPrenom.append(df_contributeurs.iloc[i,4])
        
        else :
            #url générale:
            df_contributeurs.iloc[i,3]="https://www.idref.fr/Sru/Solr?q=persname_t:"+NOM2+"&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
            RechercheG.append(df_contributeurs.iloc[i,3])
            
            #url NOM AND Prenom : 
            df_contributeurs.iloc[i,4] ="https://www.idref.fr/Sru/Solr?q=persname_t:("+NOM2+")&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
            RechercheNOMPrenom.append(df_contributeurs.iloc[i,4])
    
    else :
        if Prenom:

            #url générale:
            df_contributeurs.iloc[i,3]="https://www.idref.fr/Sru/Solr?q=persname_t:"+NOM+"%20"+Prenom+"&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
            RechercheG.append(df_contributeurs.iloc[i,3])

            #url NOM AND Prenom:
            df_contributeurs.iloc[i,4] ="https://www.idref.fr/Sru/Solr?q=persname_t:("+NOM+"%20AND%20"+Prenom+")&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
            RechercheNOMPrenom.append(df_contributeurs.iloc[i,4])
        
        else :
            #url générale:
            df_contributeurs.iloc[i,3]="https://www.idref.fr/Sru/Solr?q=persname_t:"+NOM+"&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
            RechercheG.append(df_contributeurs.iloc[i,3])
            
            #url NOM AND Prenom : 
            df_contributeurs.iloc[i,4] ="https://www.idref.fr/Sru/Solr?q=persname_t:("+NOM+")&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
            RechercheNOMPrenom.append(df_contributeurs.iloc[i,4])
    
    

In [9]:
df_contributeurs

,NOM (Prenom),NOM,Prenom,Recherche_G_t,Recherche_NomPrenom_t
0,ACHALME (P.),ACHALME,P.,https://www.idref.fr/Sru/Solr?q=persname_t:ACH...,https://www.idref.fr/Sru/Solr?q=persname_t:(AC...
1,ADDE (R.),ADDE,R.,https://www.idref.fr/Sru/Solr?q=persname_t:ADD...,https://www.idref.fr/Sru/Solr?q=persname_t:(AD...
2,AHMED (E.),AHMED,E.,https://www.idref.fr/Sru/Solr?q=persname_t:AHM...,https://www.idref.fr/Sru/Solr?q=persname_t:(AH...
3,ALARY (A.),ALARY,A.,https://www.idref.fr/Sru/Solr?q=persname_t:ALA...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...
4,ALLAIN (Jacques),ALLAIN,Jacques,https://www.idref.fr/Sru/Solr?q=persname_t:ALL...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...
...,...,...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.,https://www.idref.fr/Sru/Solr?q=persname_t:YVE...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...
659,YVON (P.),YVON,P.,https://www.idref.fr/Sru/Solr?q=persname_t:YVO...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...
660,ZABOLOTNY,ZABOLOTNY,,https://www.idref.fr/Sru/Solr?q=persname_t:ZAB...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...
661,ZAMMIT (TH.),ZAMMIT,TH.,https://www.idref.fr/Sru/Solr?q=persname_t:ZAM...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...


## 3. Chercher s'il existe au moins une notice associée à l'url idref

Méthode : pour chaque url chercher la valeur de l'attribut numFound dans l'elt <result>
\    
`<result name="response" numFound="0" start="0"/>`

In [10]:
df_contributeurs = df_contributeurs.reindex(columns = df_contributeurs.columns.tolist() + ['Recherche_G-nbr','Recherche_NomPrenom-nbr'])    

In [11]:
df_contributeurs

,NOM (Prenom),NOM,Prenom,Recherche_G_t,Recherche_NomPrenom_t,Recherche_G-nbr,Recherche_NomPrenom-nbr
0,ACHALME (P.),ACHALME,P.,https://www.idref.fr/Sru/Solr?q=persname_t:ACH...,https://www.idref.fr/Sru/Solr?q=persname_t:(AC...,NaN,NaN
1,ADDE (R.),ADDE,R.,https://www.idref.fr/Sru/Solr?q=persname_t:ADD...,https://www.idref.fr/Sru/Solr?q=persname_t:(AD...,NaN,NaN
2,AHMED (E.),AHMED,E.,https://www.idref.fr/Sru/Solr?q=persname_t:AHM...,https://www.idref.fr/Sru/Solr?q=persname_t:(AH...,NaN,NaN
3,ALARY (A.),ALARY,A.,https://www.idref.fr/Sru/Solr?q=persname_t:ALA...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,NaN,NaN
4,ALLAIN (Jacques),ALLAIN,Jacques,https://www.idref.fr/Sru/Solr?q=persname_t:ALL...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,NaN,NaN
...,...,...,...,...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.,https://www.idref.fr/Sru/Solr?q=persname_t:YVE...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,NaN,NaN
659,YVON (P.),YVON,P.,https://www.idref.fr/Sru/Solr?q=persname_t:YVO...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,NaN,NaN
660,ZABOLOTNY,ZABOLOTNY,,https://www.idref.fr/Sru/Solr?q=persname_t:ZAB...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,NaN,NaN
661,ZAMMIT (TH.),ZAMMIT,TH.,https://www.idref.fr/Sru/Solr?q=persname_t:ZAM...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,NaN,NaN


# 3. Extraire le nombre d'occurences du nom selon le type de recherche

In [12]:
def IdRef_numfound_auteur (col):
    list_numfound_IdRef = []

    for url in col:
        r = requests.get(url)
        xml = r.text
        #print(r.text)

        soup = BeautifulSoup(xml,'lxml')
        eltResult = soup.find_all('result')

        for att in eltResult:
            attnumFound = att.get("numfound")

        list_numfound_IdRef.append(attnumFound)
    
    return list_numfound_IdRef

#<result name="response" numFound="0" start="0"/>

In [13]:
col = df_contributeurs["Recherche_G_t"]
df_contributeurs["Recherche_G-nbr"] = IdRef_numfound_auteur(col)

In [14]:
df_contributeurs

,NOM (Prenom),NOM,Prenom,Recherche_G_t,Recherche_NomPrenom_t,Recherche_G-nbr,Recherche_NomPrenom-nbr
0,ACHALME (P.),ACHALME,P.,https://www.idref.fr/Sru/Solr?q=persname_t:ACH...,https://www.idref.fr/Sru/Solr?q=persname_t:(AC...,4,NaN
1,ADDE (R.),ADDE,R.,https://www.idref.fr/Sru/Solr?q=persname_t:ADD...,https://www.idref.fr/Sru/Solr?q=persname_t:(AD...,27,NaN
2,AHMED (E.),AHMED,E.,https://www.idref.fr/Sru/Solr?q=persname_t:AHM...,https://www.idref.fr/Sru/Solr?q=persname_t:(AH...,3628,NaN
3,ALARY (A.),ALARY,A.,https://www.idref.fr/Sru/Solr?q=persname_t:ALA...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,77,NaN
4,ALLAIN (Jacques),ALLAIN,Jacques,https://www.idref.fr/Sru/Solr?q=persname_t:ALL...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,364,NaN
...,...,...,...,...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.,https://www.idref.fr/Sru/Solr?q=persname_t:YVE...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,16,NaN
659,YVON (P.),YVON,P.,https://www.idref.fr/Sru/Solr?q=persname_t:YVO...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,854,NaN
660,ZABOLOTNY,ZABOLOTNY,,https://www.idref.fr/Sru/Solr?q=persname_t:ZAB...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,2,NaN
661,ZAMMIT (TH.),ZAMMIT,TH.,https://www.idref.fr/Sru/Solr?q=persname_t:ZAM...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,33,NaN


In [15]:
df_contributeurs.iloc[2,4]

'https://www.idref.fr/Sru/Solr?q=persname_t:(AHMED%20AND%20E.)&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z'

In [16]:
col = df_contributeurs["Recherche_NomPrenom_t"]
df_contributeurs["Recherche_NomPrenom-nbr"] = IdRef_numfound_auteur(col)

In [17]:
df_contributeurs

,NOM (Prenom),NOM,Prenom,Recherche_G_t,Recherche_NomPrenom_t,Recherche_G-nbr,Recherche_NomPrenom-nbr
0,ACHALME (P.),ACHALME,P.,https://www.idref.fr/Sru/Solr?q=persname_t:ACH...,https://www.idref.fr/Sru/Solr?q=persname_t:(AC...,4,0
1,ADDE (R.),ADDE,R.,https://www.idref.fr/Sru/Solr?q=persname_t:ADD...,https://www.idref.fr/Sru/Solr?q=persname_t:(AD...,27,0
2,AHMED (E.),AHMED,E.,https://www.idref.fr/Sru/Solr?q=persname_t:AHM...,https://www.idref.fr/Sru/Solr?q=persname_t:(AH...,3628,15
3,ALARY (A.),ALARY,A.,https://www.idref.fr/Sru/Solr?q=persname_t:ALA...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,77,0
4,ALLAIN (Jacques),ALLAIN,Jacques,https://www.idref.fr/Sru/Solr?q=persname_t:ALL...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,364,5
...,...,...,...,...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.,https://www.idref.fr/Sru/Solr?q=persname_t:YVE...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,16,0
659,YVON (P.),YVON,P.,https://www.idref.fr/Sru/Solr?q=persname_t:YVO...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,854,4
660,ZABOLOTNY,ZABOLOTNY,,https://www.idref.fr/Sru/Solr?q=persname_t:ZAB...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,2,2
661,ZAMMIT (TH.),ZAMMIT,TH.,https://www.idref.fr/Sru/Solr?q=persname_t:ZAM...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,33,0


# 4. Exploration des résultats

In [18]:
df_contributeurs.max()

NOM (Prenom)                                                      ZUCCARELLI
NOM                                                               ZUCCARELLI
Prenom                                                                   ÉT.
Recherche_G_t              https://www.idref.fr/Sru/Solr?q=persname_t:ZUC...
Recherche_NomPrenom_t      https://www.idref.fr/Sru/Solr?q=persname_t:(ZU...
Recherche_G-nbr                                                           99
Recherche_NomPrenom-nbr                                                    9
dtype: object

In [19]:
df_contributeurs.to_csv("IdRef_BSPE.tsv", sep="\t")

In [20]:
df_contributeurs.describe()

,NOM (Prenom),NOM,Prenom,Recherche_G_t,Recherche_NomPrenom_t,Recherche_G-nbr,Recherche_NomPrenom-nbr
count,663,663,663,663,663,663,663
unique,657,596,152,656,656,239,66
top,HENRY (A.),CLAPIER,A.,https://www.idref.fr/Sru/Solr?q=persname_t:HEN...,https://www.idref.fr/Sru/Solr?q=persname_t:(LE...,0,0
freq,2,4,86,2,2,79,373


## Pour Recherche_G

In [21]:
df_General = df_contributeurs[['NOM (Prenom)','Recherche_G-nbr']]

In [22]:
df_General["Recherche_G-nbr"].astype(int).describe()

count      663.000000
mean       758.110106
std       3328.115022
min          0.000000
25%          4.000000
50%         26.000000
75%        149.000000
max      29200.000000
Name: Recherche_G-nbr, dtype: float64

In [24]:
L_G40={}
L_Gover40={}
L_G1=[]
L_G0=[]


for i in range (len(df_contributeurs)):
    occ = int(df_contributeurs.iloc[i,5])
    if occ in range(2,40) :
        auteur = df_contributeurs.iloc[i,0]
        L_G40[auteur]=occ
    if occ >= 40 :
        auteur = df_contributeurs.iloc[i,0]
        L_Gover40[auteur]=occ
    elif occ == 1:
        auteur = df_contributeurs.iloc[i,0]
        L_G1.append(auteur)
    elif occ == 0:
        auteur = df_contributeurs.iloc[i,0]
        L_G0.append(auteur)
print("nombre d'auteurs ayant entre 2 et 40 occurences dansIdRef : ",len(L_G40))

print("nombre d'auteurs ayant 1 seule occurence dansIdRef : ",len(L_G1))

print("nombre d'auteurs ayant 0 occurence dansIdRef : ",len(L_G0))


print("\n\n\n")

print("liste des auteurs ayant entre 2 et 40 occurences dans IdRef",L_G40)
print("\n\n")
print("liste des auteurs ayant 1 seule occurence dans IdRef",L_G40)
print("\n\n")
print("liste des auteurs ayant 0 occurence dans IdRef",L_G0)


nombre d'auteurs ayant entre 2 et 40 occurences dansIdRef :  267
nombre d'auteurs ayant 1 seule occurence dansIdRef :  25
nombre d'auteurs ayant 0 occurence dansIdRef :  79




liste des auteurs ayant entre 2 et 40 occurences dans IdRef {'ACHALME (P.)': 4, 'ADDE (R.)': 27, 'ANFREVILLE (Léon D’)': 5, 'ARDOIN (E.)': 36, 'ARGAUD (R.)': 21, 'ARLO (J.)': 16, 'ARROUX': 2, 'ARZT (L.)': 13, 'ATHIAS (G.)': 22, 'AUDAIN (L.)': 8, 'BABAULT (G.)': 18, 'BACMANN (A.)': 3, 'BALOZET (L.)': 2, 'BARTET (A.)': 13, 'BARTET()': 13, 'BAUCHE (J.)': 30, 'BAUJEAN (R.)': 3, 'BAYEUX (R.)': 29, 'BEAUJEAN (R.)': 39, 'BÉCLÊRE': 4, 'BEGUET (M.)': 18, 'BEGUET()': 18, 'BELLEVAL (G.)': 12, 'BENOIT-GONIN': 9, 'BEQUAERT (J.)': 4, 'BIENASSIS (E.)': 6, 'BIJON (R.)': 17, 'BILLIEMAZ (A.)': 6, 'BIZARD': 27, 'BLAIZOT (L.)': 31, 'BLANQUIER (H.)': 4, 'BOSSAVY (F.)': 6, 'BOTREAU-ROUSSEL': 6, 'BOUILLIEZ (M.)': 6, 'BRÉMONT (E.)': 33, 'BRIMONT (E.)': 10, 'BRODEN (A.)': 4, 'BRUG (S.-L.)': 31, 'BRUMPT (Emile)': 5, 'BRUYANT (L.)': 23, '

In [25]:
####### Exploration + détaillée de ceux qui ont 1 seule occurence

dfG1_occ=pd.DataFrame(L_G1)
dfG0_occ=pd.DataFrame(L_G0)
dfG40_occ=pd.DataFrame.from_dict(L_G40,orient='index')
dfGover40_occ=pd.DataFrame.from_dict(L_Gover40,orient='index')

In [26]:
dfG40_occ

,0
ACHALME (P.),4
ADDE (R.),27
ANFREVILLE (Léon D’),5
ARDOIN (E.),36
ARGAUD (R.),21
...,...
YOFÉ (H.),4
YVERNAULT (A.),16
ZABOLOTNY,2
ZAMMIT (TH.),33


In [35]:
dfG40_occ.describe()

,0
count,267.000000
mean,13.262172
std,10.393688
min,2.000000
25%,4.000000
50%,10.000000
75%,20.500000
max,39.000000


In [28]:
dfGover40_occ.nlargest(15,[0])

,0
ROBERT,29200
ROBERT (Léopold),29200
ROBERT (M.J.F.),29200
R. ASSEO),29070
DAVID,29017
MARTIN (Gustave),18460
MARTIN (Louis),18460
MARTIN (Noël),18460
RICHARD (G.),18034
BERNARD (P. NOËL),17175


## Groupby : NOM(Prenom)

In [29]:
df_contributeurs["Recherche_NomPrenom-nbr"].astype(int).describe()

count      663.000000
mean       191.327300
std       2053.232144
min          0.000000
25%          0.000000
50%          0.000000
75%          2.000000
max      29200.000000
Name: Recherche_NomPrenom-nbr, dtype: float64

In [31]:
L_NP40={}
L_NPover40={}
L_NP1=[]
L_NP0=[]


for i in range (len(df_contributeurs)):
    occ = int(df_contributeurs.iloc[i,6])
    if occ in range(2,40) :
        auteur = df_contributeurs.iloc[i,0]
        L_NP40[auteur]=occ
    if occ >= 40 :
        auteur = df_contributeurs.iloc[i,0]
        L_NPover40[auteur]=occ
    elif occ == 1:
        auteur = df_contributeurs.iloc[i,0]
        L_NP1.append(auteur)
    elif occ == 0:
        auteur = df_contributeurs.iloc[i,0]
        L_NP0.append(auteur)
print("nombre d'auteurs ayant entre 2 et 40 occurences dansIdRef : ",len(L_NP40))

print("nombre d'auteurs ayant 1 seule occurence dansIdRef : ",len(L_NP1))

print("nombre d'auteurs ayant 0 occurence dansIdRef : ",len(L_NP0))

print("nombre d'auteurs + de 40 occurencs dansIdRef : ",len(L_NPover40))

print("\n\n\n")

print("liste des auteurs ayant entre 2 et 40 occurences dans IdRef",L_NP40)
print("\n\n")
print("liste des auteurs ayant 1 seule occurence dans IdRef",L_NP1)
print("\n\n")
print("liste des auteurs ayant 0 occurence dans IdRef",L_NP0)
print("\n\n")
print("Liste des auteurs + de 40 occurencs dansIdRef : ",L_NPover40)


nombre d'auteurs ayant entre 2 et 40 occurences dansIdRef :  147
nombre d'auteurs ayant 1 seule occurence dansIdRef :  106
nombre d'auteurs ayant 0 occurence dansIdRef :  373
nombre d'auteurs + de 40 occurencs dansIdRef :  35




liste des auteurs ayant entre 2 et 40 occurences dans IdRef {'AHMED (E.)': 15, 'ALLAIN (Jacques)': 5, 'ARIAS (J.)': 7, 'ARROUX': 2, 'AUBERT (P.)': 4, 'AUGÉ (J.)': 2, 'BALFOUR (A.)': 3, 'BARTET()': 13, 'BASILE (C.)': 3, 'BÉCLÊRE': 4, 'BEGUET()': 18, 'BENOIT-GONIN': 9, 'BETTENCOURT (A.)': 3, 'BIZARD': 27, 'BLANC (G.)': 12, 'BLANC (GEORGES)': 14, 'BLANCHARD (Camille)': 2, 'BLANCHARD (M.)': 10, 'BONNET (A.)': 11, 'BOTREAU-ROUSSEL': 6, 'BOUFFARD (Gustave)': 2, 'BOUTIN (P.)': 2, 'BRAULT (J.)': 3, 'BRUN (G.)': 2, 'CALMETTE (A.)': 2, 'CASTELLANI (A.)': 3, 'CHATTON (Edouard)': 2, 'CIUCA (M.)': 2, 'CLAPIER': 24, 'COLLIN (L.)': 6, 'COMTE (C.)': 30, 'CONOR (A.)': 2, 'CORDIER (E.)': 3, 'COUTEAUD': 4, 'COUTO (M.)': 5, 'CUENOD (A.)': 2, 'DANG VAN CUONG': 30, 'DE MOUZON': 5, 

In [32]:
####### Exploration + détaillée de ceux qui ont 1 seule occurence

dfNP1_occ=pd.DataFrame(L_NP1)
dfNP0_occ=pd.DataFrame(L_NP0)
dfNP40_occ=pd.DataFrame.from_dict(L_NP40,orient='index')
dfNPover40_occ=pd.DataFrame.from_dict(L_NPover40,orient='index')

In [34]:
dfNP40_occ.describe()

,0
count,147.000000
mean,9.340136
std,9.150564
min,2.000000
25%,3.000000
50%,5.000000
75%,13.000000
max,38.000000


In [ ]:
df_NP=df_contributeurs.groupby(by="NOM (Prenom)").count()

In [ ]:
df_NP

### A) Pour l'url "générale" 

In [ ]:
#Le maximum d'occurences d'un même nom est de 99.061
#On  535 noms de personnes qui n'apparaissent pas dans IdRef

print("le maximum d'occurences pour un même url IdRef est : ",df_contributeurs["Recherche_G-nbr"].max())
df_G=df_contributeurs.groupby(by="Recherche_G-nbr").count()

In [ ]:
df_G.sort_values(by="NOM")
df_G.sort_index ## ON voit que seuls 90 noms parai

In [ ]:
df_G.describe()

In [ ]:
df_G_zero=df_contributeurs.loc[df_contributeurs["Recherche_G-nbr"] == "0"]

df_G_zero

In [ ]:
#Nbr d'absence d'occurence:
df_G_zero.size #300 / 1500 

In [ ]:
df_G.count(0)

In [ ]:
df_G

In [ ]:
df_G.plot.hist()

### B) Pour l'url "NOMPrenom" 

In [ ]:
groups = df_contributeurs.groupby(by="Recherche_NomPrenom-nbr").count()
print(type(groups))

In [ ]:
groups.value_counts
#Seulement 69 types d'occurences différentes

In [ ]:
groups.iloc[2,1]